
Linear Regression Using TensorFlow
==================================

In this notebook, we will go through the steps for performing Linear Regression on using TensorFlow. We will start with a simple problem and then scale it up using a real data example. 

---
Loading Libraries and Dependencies
----------------------------------

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

---
Part 1: Simple Problem
--------------------

We all know that linear regression can be written as 
       
       y = x*W + b 
       where x = Data Points
             y = Outcome variable
             W = Weights
             b = Bias terms

In this problem we will randomly generate the values of 'x' and then generate an equation of 
    y = 3*x + 10

We will then train our tensorflow model to calculate the values of W (which is 3) and b (which is 10).

#### Initialize the variables

In [2]:
# Initialize the x and y variables
x = tf.placeholder("float")
y = tf.placeholder("float")

# Initialize the weight and bias terms
W = tf.Variable([0.0], "float")
b = tf.Variable([0.0], "float")

# Generate the y' value which is the outcome of our equation
y_model = tf.mul(x, W) + b

# Define the cost function -- this is just the squared error of the original y and the calculated y
cost = tf.square(y - y_model)

# We now define the optimizer wiz GradientDescent with a learning rate of 0.5. We want it to minimize the cost
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(cost)

# Now that we have all the variables needed, we will initialize them
init = tf.initialize_all_variables()


# Create an empty list to store error
error = []
# Train our model
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(1000):
        x_vals = np.random.rand()
        y_vals = 3*x_vals + 10
        _, err = sess.run([optimizer, cost], feed_dict={x:x_vals, y:y_vals})
        error.append(err)
    
    W_ = sess.run(W)
    b_ = sess.run(b)
    print "The final model trained is y = x*{0} + {1}".format(W_, b_)    


The final model trained is y = x*[ 3.00000072] + [ 10.00000095]


As we can see our model has accurately found out the values of W and b as 3 and 10 respectively. This is exactly the value that we had provided as our training data.

For the next step, we will use a dataset to generate the model. This will be a bit more complex as we will have more variables in x, thus resulting in a bigger W matrix and b array.
    

---
Part 2: Complex Problem
-----------------------

In this section we will load some data and then run a linear regression on that data using TensorFlow. We will use the same steps as above but this time it will be with a larger weight matrix and a larger bias array.

In [159]:
# We will use the Boston Data set that comes with sklearn
from sklearn.datasets import load_boston
boston = load_boston(return_X_y=True)

# Separating the target and the data
data = boston[0]
target = boston[1]

# Normalize the data
data_norm = (data - np.mean(data, axis = 0)) / np.var(data, axis = 0)

# Let us initialize the variables like above
x = tf.placeholder("float", shape = [None, 13])
y = tf.placeholder("float", shape = [None, 1])

# Initialize weights and bias
W = tf.Variable(tf.ones([13, 1]))
b = tf.Variable(tf.ones([1]))

# Define the model for our training
y_model = tf.matmul(x, W) + b

# Define the cost function to minimize
cost = tf.reduce_mean(tf.square(y - y_model))

# Start our optimizer
optimizer = tf.train.AdagradOptimizer(0.5).minimize(cost)

# Start all variables
init = tf.initialize_all_variables()

error = []
# Start the training process
with tf.Session() as sess:
    sess.run(init)
    for i in range(20000):
        _, err = sess.run([optimizer, cost], feed_dict={x: data_norm, y: target.reshape(506, -1)})
        error.append(err)
        
        if i % 1000 == 0:
            print "Error at round {0} is {1}".format(i, err)
        
    W_ = sess.run(W)
    b_ = sess.run(b)
    
    print "The final model trained is Weights: {0} + Bias : {1}".format(W_, b_)

Error at round 0 is 683.422912598
Error at round 1000 is 26.8561878204
Error at round 2000 is 22.9068012238
Error at round 3000 is 22.6324672699
Error at round 4000 is 22.5379657745
Error at round 5000 is 22.4826049805
Error at round 6000 is 22.4434165955
Error at round 7000 is 22.4130401611
Error at round 8000 is 22.3882846832
Error at round 9000 is 22.3674411774
Error at round 10000 is 22.3494987488
Error at round 11000 is 22.3337993622
Error at round 12000 is 22.3198757172
Error at round 13000 is 22.3074131012
Error at round 14000 is 22.2961483002
Error at round 15000 is 22.2858963013
Error at round 16000 is 22.2764873505
Error at round 17000 is 22.2678070068
Error at round 18000 is 22.2597446442
Error at round 19000 is 22.2522258759
The final model trained is Weights: [[ -7.91777563e+00]
 [  2.05953579e+01]
 [ -2.57939720e+00]
 [  1.92973912e-01]
 [ -2.50224799e-01]
 [  1.90705466e+00]
 [  6.81487098e-02]
 [ -6.58469296e+00]
 [  1.19433498e+01]
 [ -5.53810501e+01]
 [ -4.57343435e+0

We can now compare these finding with sklear's linear model to check if our GradientDescent model has reached its minima. Sklearn using a closed form solution for Linear regression and hence is quicker.

In [143]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(data_norm, target)

print lm.coef_.reshape(13, -1), lm.intercept_

[[ -7.90475175e+00]
 [  2.51861942e+01]
 [  9.79835276e-01]
 [  1.73104308e-01]
 [ -2.38482722e-01]
 [  1.87458334e+00]
 [  5.93933942e-01]
 [ -6.53060495e+00]
 [  2.31278565e+01]
 [ -3.49519998e+02]
 [ -4.46004154e+00]
 [  7.81295547e+01]
 [ -2.67430880e+01]] 22.5328063241
